In [1]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

In [2]:
ru_emb = KeyedVectors.load_word2vec_format('/kaggle/input/ru-uk-emb/data/cc.ru.300.vec')
uk_emb = KeyedVectors.load_word2vec_format('/kaggle/input/ru-uk-emb/data/cc.uk.300.vec')

In [4]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0000001192092896),
 ('июль', 0.9383152723312378),
 ('сентябрь', 0.9240029454231262),
 ('июнь', 0.9222575426101685),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087710380554),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545795679092407),
 ('февраль', 0.8401415944099426)]

In [5]:
ru_emb.most_similar([uk_emb["серпень"]], topn=10)

[('Недопустимость', 0.24435284733772278),
 ('конструктивность', 0.23293080925941467),
 ('офор', 0.23256804049015045),
 ('deteydlya', 0.230317160487175),
 ('пресечении', 0.22632379829883575),
 ('одностороннего', 0.22608885169029236),
 ('подход', 0.22305874526500702),
 ('иболее', 0.2200372815132141),
 ('2015Александр', 0.21872766315937042),
 ('конструктивен', 0.21796567738056183)]

In [6]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [7]:
uk_ru_train, X_train, Y_train = load_word_pairs("/kaggle/input/ru-uk-emb/data/ukr_rus.train.txt")

In [8]:
print(np.array(uk_ru_train).shape)
print(X_train.shape)
print(Y_train.shape)

(1840, 2)
(1840, 300)
(1840, 300)


In [9]:
uk_ru_test, X_test, Y_test = load_word_pairs("/kaggle/input/ru-uk-emb/data/ukr_rus.test.txt")

In [10]:
print(np.array(uk_ru_test).shape)
print(X_test.shape)
print(Y_test.shape)

(387, 2)
(387, 300)
(387, 300)


In [11]:
from sklearn.linear_model import LinearRegression

In [12]:
mapping = LinearRegression(fit_intercept=False)
mapping.fit(X_train, Y_train)

LinearRegression(fit_intercept=False)

In [13]:
print(mapping.coef_.shape)

(300, 300)


In [14]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8541285991668701),
 ('июнь', 0.8411202430725098),
 ('март', 0.8396994471549988),
 ('сентябрь', 0.8359869718551636),
 ('февраль', 0.832929790019989),
 ('октябрь', 0.8311845660209656),
 ('ноябрь', 0.8278923630714417),
 ('июль', 0.823452889919281),
 ('август', 0.8120501637458801),
 ('декабрь', 0.803900420665741)]

In [15]:
def precision(pairs, mapped_vectors, topn=1):
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        possible_translations = [word for (word, score) in ru_emb.most_similar(mapped_vectors[i], topn=topn)]
        if ru in possible_translations:
            num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val

In [16]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [17]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [18]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

print(precision_top1)
print(precision_top5)

0.6356589147286822
0.813953488372093


In [19]:
def learn_transform(X_train, Y_train):
    U, S, VT = np.linalg.svd(np.matmul(X_train.T, Y_train))
    return np.matmul(U, VT)

In [20]:
W = learn_transform(X_train, Y_train)

In [21]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8237906694412231),
 ('сентябрь', 0.8049710988998413),
 ('март', 0.802565336227417),
 ('июнь', 0.8021841645240784),
 ('октябрь', 0.8001735210418701),
 ('ноябрь', 0.7934483289718628),
 ('февраль', 0.7914119958877563),
 ('июль', 0.7908107042312622),
 ('август', 0.7891014814376831),
 ('декабрь', 0.7686371207237244)]

In [22]:
precision_top1 = precision(uk_ru_test, np.matmul(X_test, W))
precision_top5 = precision(uk_ru_test, np.matmul(X_test, W), 5)

print(precision_top1)
print(precision_top5)

0.6537467700258398
0.8242894056847545


# Ukrainian-Russian translator

In [23]:
with open("/kaggle/input/ru-uk-emb/data/fairy_tale.txt", "r") as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]

In [29]:
def translate(sentence):
    words = sentence.split(' ')
    result = []
    
    for word in words:
        if word not in uk_emb:
            result.append(word)
            continue
        mapped_vector = [np.matmul(uk_emb[word], W)]
        translation = ru_emb.most_similar(mapped_vector, topn=1)[0][0]
        result.append(translation)

    return ' '.join(result)

In [30]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

In [33]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

src: лисичка - сестричка і вовк - панібрат
dst: лисичка – сестричка и волк – панібрат

src: як була собі лисичка та зробила хатку, та й живе. а це приходять холоди. от лисичка замерзла та й побігла в село вогню добувать, щоб витопити. прибігає до одної баби та й каже:
dst: как была себе лисичка и сделала хатку, и и живе. а оно приходят холоди. из лисичка замерзла и и побежала во село огня добувать, чтобы витопити. прибегает к одной бабы и и каже:

src: — здорові були, бабусю! з неділею... позичте мені огню, я вам одслужу.
dst: — здоровые були, бабусю! со неділею... позичте мне огню, мной тебе одслужу.

src: — добре, — каже, — лисичко - сестричко. сідай погрійся трохи, поки я пиріжечки повибираю з печі!
dst: — добре, — каже, — лисичко – сестричко. садись погрійся трохи, пока мной пирожки повибираю со печі!

src: а баба макові пиріжки пекла. от баба вибирає пиріжки та на столі кладе, щоб прохололи; а лисичка підгляділа та за пиріг, та з хати... виїла мачок із середини, а туди напхала смі